Made changes.

In [ ]:
# ✅ Install Required Packages
!pip install sounddevice scipy google-generativeai gtts requests

# ✅ Imports
import sounddevice as sd
from scipy.io.wavfile import write
import google.generativeai as genai
import requests
from gtts import gTTS
import os

# ✅ API Keys (replace with real ones)
HF_API_KEY = os.getenv("hugging_Face_API")  # Replace with valid Hugging Face key
GOOGLE_API_KEY = os.getenv("google_API")    # Replace with valid Gemini API key

# ✅ Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)

# ✅ Step 1: Record audio
def record_audio(filename="input.wav", duration=5, fs=16000):
    print("🎤 Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    write(filename, fs, recording)
    print(f"✅ Audio saved to {filename}")
    return filename

# ✅ Step 2: Transcribe with Hugging Face Whisper
def transcribe_with_whisper(audio_path="input.wav"):
    url = "https://api-inference.huggingface.co/models/openai/whisper-large-v3"
    headers = {
        "Authorization": f"Bearer {HF_API_KEY}",
        "Content-Type": "audio/wav"
    }

    with open(audio_path, "rb") as f:
        audio_data = f.read()

    print("📝 Transcribing audio with Whisper...")
    response = requests.post(url, headers=headers, data=audio_data)

    if response.status_code == 200:
        result = response.json()
        text = result.get("text", "").strip()
        print("✅ Transcription:", text)
        return text
    else:
        print(f"❌ Whisper API Error {response.status_code}: {response.text}")
        return None

# ✅ Step 3: Get AI response from Gemini
def get_google_ai_response(prompt):
    try:
        model = genai.GenerativeModel("models/gemini-1.5-flash-latest")
        response = model.generate_content(prompt)
        print("🤖 Gemini Response:", response.text)
        return response.text
    except Exception as e:
        print("❌ Gemini Error:", str(e))
        return None

# ✅ Step 4: Convert AI response to speech
def speak_text(text, lang="en"):
    tts = gTTS(text=text, lang=lang)
    tts.save("response.mp3")
    if os.name == 'nt':  # Windows
        os.system("start response.mp3")
    elif os.name == 'posix':  # Mac or Linux
        os.system("afplay response.mp3")  # Mac
        # os.system("mpg123 response.mp3")  # Linux

# ✅ MAIN: Continuous Loop until "exit"
if __name__ == "__main__":
    print("🎯 Say 'exit' anytime to stop.")
    while True:
        audio_file = record_audio()
        transcribed_text = transcribe_with_whisper(audio_file)

        if not transcribed_text:
            continue

        # Check if user wants to exit
        if transcribed_text.lower() == "exit":
            print("👋 Exiting...")
            break

        ai_reply = get_google_ai_response(transcribed_text)
        if ai_reply:
            speak_text(ai_reply)


🎯 Say 'exit' anytime to stop.
🎤 Recording...
✅ Audio saved to input.wav
📝 Transcribing audio with Whisper...
✅ Transcription: Hello.
🤖 Gemini Response: Hello there! How can I help you today?

🎤 Recording...
✅ Audio saved to input.wav
📝 Transcribing audio with Whisper...
✅ Transcription: Okay.
🤖 Gemini Response: Okay.  What can I help you with?

🎤 Recording...
✅ Audio saved to input.wav
📝 Transcribing audio with Whisper...
✅ Transcription: Okay.
🤖 Gemini Response: Okay.  What can I do for you?

🎤 Recording...
✅ Audio saved to input.wav
📝 Transcribing audio with Whisper...
✅ Transcription: Okay.
🤖 Gemini Response: Okay.  What can I do for you?

🎤 Recording...
✅ Audio saved to input.wav
📝 Transcribing audio with Whisper...
✅ Transcription: Okay.
❌ Gemini Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.goog